# Form Recognizer サービスを使用した領収書の分析

![A robot holding a receipt](./images/receipt_analysis.jpg)

コンピュータビジョンの人工知能（AI）分野では、印刷された文書や手書きの文書を読み取るために光学式文字認識（OCR）がよく使われます。多くの場合、文書からテキストを抽出して、さらなる処理や分析に使用できるフォーマットに変換します。

より高度なOCRのシナリオは、注文書や請求書などのフォームから、フォームのフィールドが何を表しているかを意味的に理解して情報を抽出することです。**Form Recognizer**サービスは、このようなAIの課題に特化して設計されています。

## 領収書の例

この例では、Form Recognizerのビルトインモデルを使って領収書を分析します。

下の**Run Cell** (&#9655;) ボタン（セルの左）をクリックして実行すると、Form Recognizerを使って分析するレシートの例が表示されます。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Form Recognizer リソースを作成する

>**補足:** Form Recognizerのサービスにアクセスするには、Cognitive ServiceリソースまたはForm Recognizerリソースのどちらかを使用します。

AzureサブスクリプションにForm Recognizerリソースを作成するには、以下の手順を行ってください。

1. 新しいブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。
   
2. **&#65291;リソースの作成** ボタンをクリックし、*Form Recognizer*を検索します。
3. サービスの一覧から **Form Recognizer** を選択します。
4. **Form Recognizer** のパネルで **作成** をクリックします。
5. **Form Recognizerの作成** 画面で、以下のように設定し **確認と作成** をクリックします。   
   
   - **サブスクリプション**: *ご自身のサプスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **リージョン**: *利用可能なリージョンを選択（例:東日本）*
    - **名前**: *ユニークな名前を入力*
    - **価格レベル**: F0

6. 作成されるまでしばらく待ちます。
7. 新しく作成したForm RecognizerサービスをAzureポータルで確認し、**キーとエンドポイント**ページで**キー1**と**エンドポイント**の値をコピーして、**YOUR_FORM_KEY**と**YOUR_FORM_ENDPOINT**を置き換えて、以下のコードセルに貼り付けます。

In [ ]:
form_key = 'd490882af98a4ba6b6038b7b47de4e9e'
form_endpoint = 'https://myformrecognizer-20211208.cognitiveservices.azure.com/'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## 領収書の分析

これで、Form Recognizerを使ってレシートを分析する準備が整いました。

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Form Recognizerは、フォーム内のデータを解釈して、販売者の住所や電話番号、取引日時、さらには行数、小計、税額、合計金額を正しく認識していることに注目してください。

## 詳細な情報

Form Recognizerサービスの詳細については、[Azure Form Recognizer のドキュメント](https://docs.microsoft.com/ja-jp/azure/cognitive-services/form-recognizer/index)をご参照ください。